In [1]:
import numpy as np
import h5py
from tqdm.auto import tqdm
from datetime import datetime

/opt/homebrew/anaconda3/envs/research/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = "/Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_ship_663221_test.h5"

if "ds" in vars():
    ds.close()   # type: ignore

ds = h5py.File(dataset_path, "r")
for attr in ds.attrs:
    print(f"{attr}: {ds.attrs[attr]}")

author: Mark Vodyanitskiy (mvodya@icloud.com)
created_at: 2025-07-13T14:26:08.378871
filter_rules: MIN_TOTAL_POINTS=50, MIN_MOVING_POINTS=5, MIN_MAX_SPEED=20, SPEED_MOVING_MIN=10, SPEED_SANITY_MAX=800
filtered_at: 2026-01-08T06:02:47.931391
sources_count: 27555
sources_size: 439.3Gb
version: 1.0


In [3]:
# speed в десятых узла: 10 = 1.0 kn

CHUNK_ROWS = 2_000_000

MOVE_SPEED_MIN = 15        # >= 1.5 kn -> движение
STOP_SPEED_MAX = 10         # <= 1.0 kn -> стоп

STOP_CONFIRM_POINTS = 5    # сколько стоп-точек подряд нужно, чтобы считать остановился
STOP_RADIUS_M = 150        # "в одном месте": радиус в метрах для подтверждения стопа

GAP_SOFT_SEC = 2 * 60 * 60       # до 2 часов - не считаем разрывом трека
GAP_HARD_SEC = 12 * 60 * 60      # >= 12 часов - считаем новым треком
GAP_DEST_SEC = 2 * 60 * 60   # если dest сменился и пропуск >= этого порога -> новый трек

# Чтобы не делать огромные dt (если редкие сообщения)
DT_CLIP_SEC = 30 * 60


In [ ]:
def iter_day_datasets(ds: h5py.File):
    gpos = ds["positions"]
    for yyyy in sorted(gpos.keys()):
        for mm in sorted(gpos[yyyy].keys()):
            for dd in sorted(gpos[yyyy][mm].keys()):
                d = gpos[yyyy][mm][dd]
                if isinstance(d, h5py.Dataset):
                    yield (yyyy, mm, dd), d

def ensure_group(h5: h5py.File, path: str) -> h5py.Group:
    g = h5
    for part in [p for p in path.split("/") if p]:
        if part not in g:
            g = g.create_group(part)
        else:
            g = g[part]
    return g

def append_rows(dst_ds: h5py.Dataset, rows: np.ndarray) -> None:
    if rows.size == 0:
        return
    old = dst_ds.shape[0]
    new = old + rows.shape[0]
    dst_ds.resize((new,))
    dst_ds[old:new] = rows

# быстрая оценка расстояния (метры) для определения, что точки в одном месте
# equirectangular approximation
def dist_m(lat1, lon1, lat2, lon2):
    # lat/lon in degrees
    R = 6371000.0
    phi1 = np.deg2rad(lat1)
    phi2 = np.deg2rad(lat2)
    dphi = phi2 - phi1
    dl = np.deg2rad(lon2 - lon1)
    x = dl * np.cos((phi1 + phi2) * 0.5)
    y = dphi
    return R * np.sqrt(x*x + y*y)

In [5]:
out_path = "/Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_tracks_20250714_v4_test.h5"

dst = h5py.File(out_path, "w")

# copy root attrs
for k, v in ds.attrs.items():
    dst.attrs[k] = v

dst.attrs["tracks_filled_at"] = datetime.utcnow().isoformat()
dst.attrs["track_rules"] = (
    f"MOVE_SPEED_MIN={MOVE_SPEED_MIN}, STOP_SPEED_MAX={STOP_SPEED_MAX}, "
    f"STOP_CONFIRM_POINTS={STOP_CONFIRM_POINTS}, STOP_RADIUS_M={STOP_RADIUS_M}, "
    f"GAP_SOFT_SEC={GAP_SOFT_SEC}, GAP_HARD_SEC={GAP_HARD_SEC}"
)

# copy /files, /zones, /ships as-is
if "files" in ds:
    ds.copy("files", dst)
if "zones" in ds:
    ds.copy("zones", dst)
if "ships" in ds:
    ds.copy("ships", dst)

# create /tracks (empty, append later)
tracks_dtype = ds["tracks"].dtype if "tracks" in ds else np.dtype([
    ("track_id", "i8"),
    ("ship_id", "i8"),
    ("start_timestamp", "i4"),
    ("end_timestamp", "i4"),
    ("start_lat", "f4"),
    ("start_lon", "f4"),
    ("end_lat", "f4"),
    ("end_lon", "f4"),
    ("points_count", "i4"),
])

tracks_ds = dst.create_dataset(
    "tracks",
    shape=(0,), maxshape=(None,),
    dtype=tracks_dtype,
    chunks=True,
    compression="gzip", compression_opts=4
)

# create positions root group
ensure_group(dst, "positions")

print("Created:", out_path)


Created: /Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_tracks_20250714_v4_test.h5


/var/folders/2c/2p1h64qd4zb1g13m_f5z4z9w0000gn/T/ipykernel_34331/3561208124.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  dst.attrs["tracks_filled_at"] = datetime.utcnow().isoformat()


In [6]:
ships_ids = ds["ships"]["ship_id"].astype(np.int64, copy=False)
N = ships_ids.size
ship_to_idx = {int(sid): i for i, sid in enumerate(ships_ids)}

print("Ships:", N)

# state arrays per ship
last_ts = np.full(N, -1, dtype=np.int32)

# Текущий активный track_id на судно (-1 если нет)
cur_track_id = np.full(N, -1, dtype=np.int64)

# Счетчик точек в текущем треке
cur_points = np.zeros(N, dtype=np.int32)

# Стартовые параметры текущего трека
trk_start_ts = np.zeros(N, dtype=np.int32)
trk_start_lat = np.zeros(N, dtype=np.float32)
trk_start_lon = np.zeros(N, dtype=np.float32)

# Последние известные координаты/время (для закрытия трека)
last_lat = np.zeros(N, dtype=np.float32)
last_lon = np.zeros(N, dtype=np.float32)

# stop confirmation state
stop_cnt = np.zeros(N, dtype=np.int16)
stop_ref_lat = np.zeros(N, dtype=np.float32)
stop_ref_lon = np.zeros(N, dtype=np.float32)
stop_active = np.zeros(N, dtype=np.bool_)   # "мы в режиме подтверждения остановки"
last_destination = np.zeros(N, dtype="S64")
have_destination = np.zeros(N, dtype=np.bool_)

# Глобальный track id генератор
next_track_id = 1

print("State initialized.")


Ships: 372403
State initialized.


In [7]:
def close_track_for_ship(i: int):
    """
    If ship i has an active track, append a row to /tracks and clear active state.
    Uses last_* as end point.
    """
    tid = int(cur_track_id[i])
    if tid < 0:
        return

    row = np.zeros((1,), dtype=tracks_ds.dtype)
    row["track_id"][0] = tid
    row["ship_id"][0] = int(ships_ids[i])
    row["start_timestamp"][0] = int(trk_start_ts[i])
    row["end_timestamp"][0] = int(last_ts[i])
    row["start_lat"][0] = float(trk_start_lat[i])
    row["start_lon"][0] = float(trk_start_lon[i])
    row["end_lat"][0] = float(last_lat[i])
    row["end_lon"][0] = float(last_lon[i])
    row["points_count"][0] = int(cur_points[i])

    append_rows(tracks_ds, row)

    # clear track state
    cur_track_id[i] = -1
    cur_points[i] = 0


In [ ]:
# Посчитаем общий размер для tqdm
days = []
total_rows = 0
for key, day_ds in iter_day_datasets(ds):
    days.append((key, day_ds))
    total_rows += int(day_ds.shape[0])

p = tqdm(total=total_rows, desc="Fill track_id + write new HDF5", unit="rows")

for (yyyy, mm, dd), day_src in days:
    p.set_postfix_str(f"{yyyy}-{mm}-{dd}")

    # create dst day dataset (resizable)
    g = ensure_group(dst, f"positions/{yyyy}/{mm}")
    day_dst = g.create_dataset(
        dd,
        shape=(0,), maxshape=(None,),
        dtype=day_src.dtype,
        chunks=True,
        compression="gzip", compression_opts=4
    )

    n = int(day_src.shape[0])
    for start in range(0, n, CHUNK_ROWS):
        end = min(n, start + CHUNK_ROWS)
        chunk = day_src[start:end]

        # output chunk = copy + overwrite track_id
        out = chunk.copy()
        out["track_id"] = -1  # default

        # chunk fields
        ship = chunk["ship_id"].astype(np.int64, copy=False)
        ts   = chunk["timestamp"].astype(np.int32, copy=False)
        sp   = chunk["speed"].astype(np.int32, copy=False)
        lat  = chunk["lat"].astype(np.float32, copy=False)
        lon  = chunk["lon"].astype(np.float32, copy=False)
        dest = chunk["destination"]  # bytes S64

        for j in range(chunk.shape[0]):
            sid = int(ship[j])
            i = ship_to_idx.get(sid, -1)
            if i < 0:
                continue

            t  = int(ts[j])
            s  = int(sp[j])
            la = float(lat[j])
            lo = float(lon[j])
            cur_dest = dest[j]  # bytes

            # 1) dt / gaps / update last
            prev = int(last_ts[i])
            if prev >= 0:
                dt = t - prev
                if dt <= 0:
                    # time went backwards or equal => just update last and continue logic
                    dt = 0
                else:
                    # clip moderate dt (optional) but keep real "big gap" detection
                    if dt > DT_CLIP_SEC and dt < GAP_SOFT_SEC:
                        dt = DT_CLIP_SEC

                    # HARD GAP => end current track unconditionally (if any)
                    if dt >= GAP_HARD_SEC:
                        close_track_for_ship(i)
                        stop_cnt[i] = 0
                        stop_active[i] = False

                # update last point
                last_ts[i]  = t
                last_lat[i] = la
                last_lon[i] = lo
            else:
                # first seen point
                dt = 0
                last_ts[i]  = t
                last_lat[i] = la
                last_lon[i] = lo

            # 2) classify point
            is_moving     = (s >= MOVE_SPEED_MIN)
            is_stop_point = (s <= STOP_SPEED_MAX)

            # 3) destination change logic (AFTER dt is known)
            #    rule: don't cut on-the-fly, only:
            #      a) stop-ish (stop point or stop_active) OR
            #      b) dest_changed + dt >= GAP_DEST_SEC
            dest_changed = False
            if have_destination[i]:
                dest_changed = (cur_dest != last_destination[i])
            else:
                have_destination[i] = True

            # always keep last_destination current
            last_destination[i] = cur_dest

            if dest_changed and (cur_track_id[i] >= 0):
                if is_stop_point or stop_active[i]:
                    close_track_for_ship(i)
                    stop_cnt[i] = 0
                    stop_active[i] = False
                elif dt >= GAP_DEST_SEC:
                    close_track_for_ship(i)
                    stop_cnt[i] = 0
                    stop_active[i] = False
                else:
                    # moving without meaningful gap -> ignore
                    pass

            # 4) stop confirmation (can end track)
            #    NOTE: if we are moving -> reset stop state
            if is_moving:
                stop_cnt[i] = 0
                stop_active[i] = False
            elif is_stop_point:
                if not stop_active[i]:
                    stop_active[i] = True
                    stop_cnt[i] = 1
                    stop_ref_lat[i] = la
                    stop_ref_lon[i] = lo
                else:
                    d = dist_m(stop_ref_lat[i], stop_ref_lon[i], la, lo)
                    if d <= STOP_RADIUS_M:
                        stop_cnt[i] += 1
                    else:
                        stop_cnt[i] = 1
                        stop_ref_lat[i] = la
                        stop_ref_lon[i] = lo

                if stop_cnt[i] >= STOP_CONFIRM_POINTS:
                    close_track_for_ship(i)
                    # keep stop_active True (optional) — так трек стартанёт только при движении
                    # stop_cnt можно оставить

            # 5) start track on movement (ONLY here)
            if is_moving and (cur_track_id[i] < 0):
                cur_track_id[i] = next_track_id
                next_track_id += 1

                trk_start_ts[i]  = t
                trk_start_lat[i] = la
                trk_start_lon[i] = lo
                cur_points[i] = 0

            # 6) assign track_id to current point
            tid = int(cur_track_id[i])
            out["track_id"][j] = tid if tid >= 0 else -1
            if tid >= 0:
                cur_points[i] += 1

        # write out chunk
        append_rows(day_dst, out)
        p.update(end - start)

p.close()

# close tail tracks
for i in range(N):
    if cur_track_id[i] >= 0:
        close_track_for_ship(i)

dst.flush()
print("Done. tracks:", tracks_ds.shape[0], "next_track_id:", next_track_id)
print("Output:", out_path)


Fill track_id + write new HDF5: 100%|██████████| 21182/21182 [00:00<00:00, 45473.10rows/s, 2025-07-10]


Done. tracks: 113 next_track_id: 114
Output: /Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_tracks_20250714_v4_test.h5


In [9]:
dst.close()
print("Closed:", out_path)


Closed: /Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_tracks_20250714_v4_test.h5
